# Object tracking

### Importing the libraries

In [16]:
from collections import deque
import numpy as np
import cv2
import imutils
import time

### Setting the lower and upper bound of color for the object
You can use the code at the end to find your hsv combination for other color

In [17]:
greenLower = (29, 86, 6)
greenUpper = (64, 255, 255)

### Declaring a deque

In [18]:
pts = deque(maxlen=20)

### Creating a video stream object

In [19]:
vs = cv2.VideoCapture(0)
time.sleep(2.0)

### Looping throught each frame

In [20]:
while True:
    
    # Reading a frame from camera
    frame = vs.read()[1]
    
    # Resizing the frame to make processing faster
    frame = imutils.resize(frame, width=600)     
    
    # Removing details from the image
    blurred = cv2.GaussianBlur(frame, (11, 11), 0) 
    
    # Converting frame from BGR to HSV
    hsv = cv2.cvtColor(blurred, cv2.COLOR_BGR2HSV) 

    # Creating a mask that only sees color between lower and upper bound
    mask = cv2.inRange(hsv, greenLower, greenUpper)

    # Reducing the size of the color mask to remove noise
    mask = cv2.erode(mask, None, iterations=2)
    
    # Increasing it again to get the same size 
    mask = cv2.dilate(mask, None, iterations=2)
    
    # Finding contours
    cnts = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)     
    cnts = cnts[1]
    
    center = None
    
     # If at least one contour is found 
    if len(cnts) > 0:    
        
        # Get the contour with maximum area
        c = max(cnts, key=cv2.contourArea)
        
        # Find the minimum enclosing circle of it
        ((x, y), radius) = cv2.minEnclosingCircle(c)
        
        # Putting the position of the object in top-left corner
        txt = str(x) + ',' + str(y)
        cv2.putText(frame,txt,(0, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2) 
        
        # Calculate moments
        M = cv2.moments(c)
        
        # Finding center using moments
        center = (int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"]))
        
        # If radius of object is greater than 10, then draw a circle around it
        if radius > 10:
            cv2.circle(frame, (int(x), int(y)), int(radius),(0, 255, 255), 2)
            cv2.circle(frame, center, 5, (0, 0, 255), -1)
            
    
    # Appending the center to deque
    pts.appendleft(center)
    
    # Looping through pts
    for i in range(1, len(pts)):
        
        # If the left two elements of the deque are None skip the current iteration
        if pts[i - 1] is None or pts[i] is None:      
            continue
        
        # Calculating the thickness of the line based on its position in the deque  
        thickness = int(np.sqrt(64/ float(i + 1)) * 2.5)
        
        # Drawing a line between the points pts[i] and pts[i-1]
        cv2.line(frame, pts[i - 1], pts[i], (0, 0, 255), thickness)
            
    # Displaying the frame
    cv2.imshow('temp',frame)
    
    # Displaying the mask
    cv2.imshow('temp1',mask)
    
    key = cv2.waitKey(1)
    
    # Break when 'q' is pressed
    if key == ord('q'):
        break
        
# Destroy all the windows
cv2.destroyAllWindows()

# Release the camera
vs.release()

#   

## Finding HSV Combination

In [7]:
def nothing(x):
    pass

vs = cv2.VideoCapture(0)

cv2.namedWindow('Colorbars')

cv2.createTrackbar("MinHue1", "Colorbars",0,179,nothing)
cv2.createTrackbar("MinSat2", "Colorbars",0,255,nothing)
cv2.createTrackbar("MinVal3", "Colorbars",0,255,nothing)

cv2.createTrackbar("MaxHue1", "Colorbars",179,179,nothing)
cv2.createTrackbar("MaxSat2", "Colorbars",255,255,nothing)
cv2.createTrackbar("MaxVal3", "Colorbars",255,255,nothing)




while True:
    img = vs.read()[1]
    
    hsv = cv2.cvtColor(img,cv2.COLOR_BGR2HSV)
        
    Max1 = cv2.getTrackbarPos("MaxHue1", "Colorbars")
    Max2 = cv2.getTrackbarPos("MaxSat2", "Colorbars")
    Max3 = cv2.getTrackbarPos("MaxVal3", "Colorbars")

    Min1 = cv2.getTrackbarPos("MinHue1", "Colorbars")
    Min2 = cv2.getTrackbarPos("MinSat2", "Colorbars")
    Min3 = cv2.getTrackbarPos("MinVal3", "Colorbars")
    
    mask = cv2.inRange(hsv, (Min1,Min2,Min3), (Max1,Max2,Max3))
        
    cv2.imshow("Thresh", mask)      
    
    key = cv2.waitKey(1)
    if key == ord('q'):
        break
        
cv2.destroyAllWindows()
vs.release()